In [ ]:
!pip install "sagemaker>=2.123.0"

In [ ]:
import sagemaker
import boto3
import numpy as np                                
import pandas as pd                               
import os                                         
from sagemaker import get_execution_role
from datetime import datetime

# Get default bucket
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/mlops-workshop'

# Get SageMaker Execution Role
role = get_execution_role()
region = boto3.Session().region_name

# SageMaker Session
sagemaker_session = sagemaker.session.Session()

# SageMaker client
sm_client = boto3.client('sagemaker')

In [ ]:
%store -r

In [ ]:
print(model_package_arn)

In [ ]:
from sagemaker.model import ModelPackage

model_package = ModelPackage(
    model_package_arn = model_package_arn, 
    role = role,
    sagemaker_session = sagemaker_session
)

In [ ]:
model_package_update_input_dict = {
    "ModelPackageArn" : model_package_arn,
    "ModelApprovalStatus" : "Approved"
}

model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)

In [ ]:
from datetime import datetime
current_time = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

endpoint_name = 'xgb-model-' + current_time

try:
    model_package.deploy(
        endpoint_name = endpoint_name,
        serverless_inference_config = ServerlessInferenceConfig(
            memory_size_in_mb = 4096,
            max_concurrency = 1
        )
    )
except Exception as e:
    print("Make sure model is in an Approved state. Navigate to the model registry UI to approve the model\n\n", e)
    raise Exception("Make sure model is in an Approved state. Navigate to the model registry UI to approve the model\n\n")

In [ ]:
from sagemaker.predictor import Predictor
xgb_predictor = Predictor(
    endpoint_name = endpoint_name, 
    serializer = sagemaker.serializers.CSVSerializer(),
    sagemaker_session = sagemaker_session
)

In [ ]:
!aws s3 cp {test_uri}/test.csv test.csv

In [ ]:
test_data = pd.read_csv('test.csv', header = None)
test_data

In [ ]:
y_true = test_data[0]
data = test_data.drop(0, axis = 1)

In [ ]:
data.head()

In [ ]:
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(data.to_numpy(), xgb_predictor)

In [ ]:
print(predictions)

In [ ]:
pd.crosstab(index=y_true, columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

In [ ]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)